<a href="https://colab.research.google.com/github/fatimazain118/Deep-Learning-Assignments/blob/main/krishNaik/Keras_Tuner_Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Keras Tuner- Decide Number of Hidden Layers And Neuron In Neural Network

In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
#from kerastuner.tuners import RandomSearch  #if from Command promp we import it then this line can be used

In [2]:
import tensorflow as tf     #To import kerastuner we have done these
from tensorflow import keras

In [3]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 98 kB 6.3 MB/s 


In [4]:
import keras_tuner as kt

In [5]:
from keras_tuner import RandomSearch    #This actually helps to decide how many no. of hidden layers are required 

In [6]:
df=pd.read_csv('https://raw.githubusercontent.com/krishnaik06/Keras-Tuner/main/Real_Combine.csv')  #from raw data copy the link

In [7]:
df.head()  #this dataset check air quatity index

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [8]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

#### Hyperparameters
1. How many number of hidden layers we should have?
2. How many number of neurons we should have in hidden layers?
3. Learning Rate 

In [9]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):      #range of the no. of hidden layer can be in the NN
        model.add(layers.Dense(units=hp.Int('units_' + str(i),  #this dense function is used to create neurons
                                            min_value=32,   #no. of neuron that can be range between 32 to 512 foe each layer, this can be changed
                                            max_value=512,  #from this for loop our RandomSearch will be selecting the best no. of hidden layers and no. of neurons
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))  #output layer
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), #.001, .0001, .00001 hp.choice means choosing one of them and hp.INT means trying with differnt diff values in this
        loss='mean_absolute_error',                                #this is used for regression pblm statement
        metrics=['mean_absolute_error'])
    return model

In [10]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',  #in case of classification pblm we'll have here accuracy
    max_trials=5,
    executions_per_trial=3,   #means 15 iteration
    directory='project',
    project_name='Air Quality Index')

In [12]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [14]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 11s]
val_mean_absolute_error: nan

Best val_mean_absolute_error So Far: nan
Total elapsed time: 00h 00m 58s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()